In [ ]:
import datasets
dataset = datasets.load_dataset("glue", "sst2", split='validation')

X_test = dataset[:len(dataset)]["sentence"]
y_test = dataset[:len(dataset)]["label"]

In [1]:
import datasets
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
import os
import random
import sys
import transformers

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    BertTokenizer,
    BertForSequenceClassification,
    RobertaTokenizer,
    RobertaForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
import wandb
import copy
import torch.nn as nn
import math
from tqdm import tqdm, trange
from sklearn.metrics import f1_score
from dataset import ready_data, TweetDataset
from transformers import EarlyStoppingCallback
#import loralib as lora
from peft import (
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
    prepare_model_for_kbit_training,
)
# 데이터셋 로드
dataset = datasets.Array3Dload_dataset("glue", "sst2")

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

# 데이터셋 인코딩
def encode(examples):
    return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=256)

encoded_dataset = dataset.map(encode, batched=True)
train_dataset = encoded_dataset['train']
valid_dataset = encoded_dataset['validation']

/home/lab/anaconda3/envs/qlora_ex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pip install safetensors

You should consider upgrading via the '/home/lab/bumjun/qlora/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import safetensors
import torch
from safetensors.torch import load_file, save_file

pt_state_dict = load_file(
    "/home/lab/bumjun/low_rank/examples/NLU/output/init_dW_B_with_svd_by_head-rank4-alpha16-seed42/model/checkpoint-1100/adapter_model.safetensors",
    device="cpu",
)
torch.save(
    pt_state_dict,
    "/home/lab/bumjun/low_rank/examples/NLU/output/init_dW_B_with_svd_by_head-rank4-alpha16-seed42/model/checkpoint-1100/adapter_model.bin",
)